## PDepPostProcessing
Have a giant mechanism you couldn't get RMG to converge on when using
PDep? Use this script to automatically divide your mechanism into sub
networks, run each individually through RMG (and Cantherm), and
unite them back into a single PDep mechanism.
When running this notebook, make sure that RMG stops automatically
after the first iteration and does not process new reactions,
e.g., by checking out the PDepPost branch under alongd/RMG-Py on GitHub:
https://github.com/alongd/RMG-Py/commits/PDepPost

written by alongd

In [1]:
from rmgpy.data.kinetics.library import LibraryReaction, KineticsLibrary
from rmgpy.data.kinetics.database import KineticsDatabase
import os, shutil, re

# The library's name to be processed, name the new PDep-only library
libraryName = ['NG_HTP','NG_HTP_PDep']

In [2]:
# Load the library to be processed

database = KineticsDatabase()
basePath = '/home/alongd/Code/RMG-database/input/kinetics/libraries/' # path to 'RMG-database/input/kinetics/libraries/'

library_file = basePath+libraryName[0]+'/reactions.py'

library = KineticsLibrary(label=libraryName[0])
library.load(library_file, database.local_context, database.global_context)

In [4]:
# Create and save a new PDep-only library
PDepLibrary = KineticsLibrary(label=libraryName[1])

j = 0
for i, entry in library.entries.iteritems():
    if entry.item.isUnimolecular():
        j += 1
        PDepLibrary.entries[j] = library.entries[i]
        if len(PDepLibrary.entries[j].item.reactants) + len(PDepLibrary.entries[j].item.products) > 3: #assert unimolecular
            print "Error, {0} is not a unimolecular reaction".format(entry.label)

path = os.getcwd()
if not os.path.exists(path+'/output/'):
    os.makedirs(path+'/output/')
for the_file in os.listdir(path+'/output/'):
    file_path = os.path.join(path+'/output/', the_file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)
PDepLibrary.save(path+'/output/output.py')

In [8]:
# functions

# Get a species and return all reactions in which it is in the mono side
def search(S):
    rxns = []
    for i, entry in PDepLibrary.entries.iteritems(): # entry = newLibrary.entries[i]
        if (S in entry.item.reactants and len(entry.item.reactants) == 1)\
            or (S in entry.item.products and len(entry.item.products) == 1):
            rxns.append(entry)
    return rxns

# decide which side in mono, and send to extractRxns()
def extractRxns(entry):
    if entry.item.isDissociation():
        rxns = search(entry.item.reactants[0])
    elif entry.item.isAssociation():
        rxns = search(entry.item.products[0])
    elif entry.item.isIsomerization():
        rxns = search(entry.item.reactants[0])
        rxns2 = search(entry.item.products[0])
        for r in rxns2:
            if not r in rxns:
                rxns.append(r)
    else:
        print "Error, reaction is niether Dissociation, Association, nor Isomerization"
    return rxns

# find isomerization reactions (other than the first reaction which was already explored)
def isomerization(rxns):
    isom = []
    for r in rxns:
        if r.item.isIsomerization() and r != rxns[0]:
            isom.append(r)
    return isom

def RMGlog(net):
    try:
        chem = open('runs/net'+str(net)+'/chemkin/chem_annotated.inp','r').readlines()
        lines = [x for x in chem[17:] if x != "\n"]
        readSpecies = False
        species = -2 # not counting CH4(1), O2(2)
        readRxns = False
        rxns = 0
        for line in lines:
            if "SPECIES" in line:
                readSpecies = True
            if "REACTIONS" in line:
                readRxns = True
            if "END" in line:
                readSpecies = readRxns = False
            if readSpecies:
                if '(' in line:
                    species += 1
            if readRxns:
                if 'Reaction index' in line:
                    rxns += 1
        print("Done simulating net{0} with {1} species and {2} reactions.\n".format(net,species,rxns))
    except:
        return 1
    return 0

def createInputFile(net):
    cwd = os.getcwd()
    if not os.path.exists(cwd+'/runs/'):
        os.mkdir(cwd+'/runs/')
    if not os.path.exists(cwd+'/runs/net{0}/'.format(net)):
        os.mkdir(cwd+'/runs/net{0}/'.format(net))

    inputFile = '''
database(
    thermoLibraries = ['BurkeH2O2','primaryThermoLibrary','NG_HTP','Chernov','Narayanaswamy'],
    reactionLibraries = [],
    seedMechanisms = ['networks/net'''+str(net)+''''],
    kineticsDepositories = [],
    kineticsFamilies = 'default',
    kineticsEstimator = 'rate rules',
)

species(
    label='CH4',
    reactive=True,
    structure=SMILES('C'),
)

species(
    label='O2',
    reactive=True,
    structure=SMILES('[O][O]'),
)


species(
    label='Ar',
    reactive=False,
    structure=adjacencyList(
                """
                1 Ar u0 p4 c0
                """),
)

simpleReactor(
    temperature=(2000,'K'),
    pressure=(1,'atm'),
    initialMoleFractions={
        "CH4": 0.015,
        "O2": 0.02,
        "Ar": 0.9,
    },
    terminationTime=(0.001,'s'),
)

simulator(
    atol=1e-16,
    rtol=1e-8,
)

model(
    toleranceKeepInEdge=0,
    toleranceMoveToCore=0.05,
    toleranceInterruptSimulation=0.05,
    maximumEdgeSpecies=300000
)

pressureDependence(
        method='modified strong collision',
        maximumGrainSize=(0.5,'kcal/mol'),
        minimumNumberOfGrains=250,
        temperatures=(1600,2200,'K',10),
        pressures=(0.5,1.5,'bar',5),
        interpolation=('Chebyshev', 6, 4),
        maximumAtoms=16,
)

options(
    units='si',
    generateOutputHTML=False,
    generatePlots=False,
    saveEdgeSpecies=False,
    saveSimulationProfiles=False,
    saveRestartPeriod=None,
)

generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=10,
    maximumOxygenAtoms=10,
    maximumNitrogenAtoms=0,
    maximumSiliconAtoms=0,
    maximumSulfurAtoms=0,
    maximumHeavyAtoms=15,
    maximumRadicalElectrons=1,
    allowSingletO2=False,
)

'''
    path = cwd+'/runs/net{0}/input.py'.format(net)
    with open(path,'w') as RMGInputFile:
        RMGInputFile.write(inputFile)
    return

In [9]:
# Extract the different sub-PDep-networks

path = basePath+'networks/'
if os.path.exists(path):
    shutil.rmtree(path) # delete 'networks' tree if exists
os.makedirs(path)

networks = [] # an array of KineticsLibrary objects
net = 0 # network counter
processed = []
proc = 0 # processed counter

for i, entry in PDepLibrary.entries.iteritems(): # entry = PDepLibrary.entries[i]
    if entry not in processed:
        rxns = extractRxns(entry)

    stillIsomerization = 1
    while stillIsomerization: # continue untill no unexplored isomerization reactions are left in the network
        isom = isomerization(rxns)
        rxns3 = []
        for iso in isom:
            rxns3 = extractRxns(iso)
        stillIsomerization = 0
        for k in rxns3:
            if k.item.isIsomerization(): stillIsomerization == 1
            if k not in processed and k not in rxns:
                rxns.append(k)
                
    if entry not in processed:
        net += 1
        j = 0
        network = KineticsLibrary(label='network {0}'.format(net))
        for r in rxns:
            if r not in processed:
                proc += 1
                processed.append(r)
            j += 1
            network.entries[j] = r
        network.save(path+'/net{0}/reactions.py'.format(net))
        library_dictionary = basePath+libraryName[0]+'/dictionary.txt'
        shutil.copyfile(library_dictionary, path+'/net{0}/dictionary.txt'.format(net))
    for r in rxns:
        if r not in processed:
            proc += 1
            processed.append(r)

print "Done.\nProcessed {0} rxns, saved {1} independent networks.".format(len(processed),net)

Done.
Processed 1719 rxns, saved 406 independent networks.


## run RMG
Careful, this will delete previous runs!!

In [10]:
import os, shutil, time

#net=406
#n=12-1

t0 = time.time()
cwd = os.getcwd()
if os.path.exists(cwd+'/runs/'):
    shutil.rmtree(cwd+'/runs/') # delete 'runs' tree if exists

errored = []
for n in xrange(net):
    network = n + 1
    print("Simulating net{0}...".format(network))
    try:
        createInputFile(network)
        os.system('python $rmg/rmg.py runs/net'+str(network)+'/input.py')
        if not os.path.exists(cwd+'/runs/net'+str(network)+'/chemkin/'):
            ok = False
            while not ok:
                time.sleep(5)
                if os.path.exists(cwd+'/runs/net'+str(network)+'/chemkin/'):
                    ok = True
        e = RMGlog(network)
        if e == 1:
            errored.append(network)
            print("ERROR reading net{0}!".format(network))
    except:
        pass

if len(errored) > 0: # print an error summary
    for e in errored:
        print("ERROR in network {0}!".format(network))

t1 = time.time() - t0
m, s = divmod(t1, 60)
h, m = divmod(m, 60)
#d, h = divmod(h, 24)
#print('\nTotal elapsed time: %02d %02d:%02d:%02d'%(d, h, m, s))
print('\nTotal elapsed time: %02d:%02d:%02d'%(h, m, s))
print('\n\n***\nAll Done.\n***')

Simulating net1...
Done simulating net1 with 2 species and 1 reactions.

Simulating net2...
Done simulating net2 with 1 species and 1 reactions.

Simulating net3...
Done simulating net3 with 3 species and 1 reactions.

Simulating net4...
Done simulating net4 with 3 species and 1 reactions.

Simulating net5...
Done simulating net5 with 4 species and 3 reactions.

Simulating net6...
Done simulating net6 with 4 species and 3 reactions.

Simulating net7...
Done simulating net7 with 3 species and 1 reactions.

Simulating net8...
Done simulating net8 with 11 species and 10 reactions.

Simulating net9...
Done simulating net9 with 4 species and 3 reactions.

Simulating net10...
Done simulating net10 with 5 species and 4 reactions.

Simulating net11...
Done simulating net11 with 8 species and 8 reactions.

Simulating net12...
Done simulating net12 with 11 species and 8 reactions.

Simulating net13...
Done simulating net13 with 7 species and 4 reactions.

Simulating net14...
Done simulating net1

Done simulating net110 with 3 species and 3 reactions.

Simulating net111...
Done simulating net111 with 2 species and 1 reactions.

Simulating net112...
Done simulating net112 with 5 species and 5 reactions.

Simulating net113...
Done simulating net113 with 5 species and 5 reactions.

Simulating net114...
Done simulating net114 with 8 species and 8 reactions.

Simulating net115...
Done simulating net115 with 11 species and 8 reactions.

Simulating net116...
Done simulating net116 with 4 species and 3 reactions.

Simulating net117...
Done simulating net117 with 8 species and 6 reactions.

Simulating net118...
Done simulating net118 with 5 species and 3 reactions.

Simulating net119...
Done simulating net119 with 3 species and 1 reactions.

Simulating net120...
Done simulating net120 with 7 species and 6 reactions.

Simulating net121...
Done simulating net121 with 7 species and 8 reactions.

Simulating net122...
Done simulating net122 with 12 species and 20 reactions.

Simulating net123

Done simulating net217 with 3 species and 1 reactions.

Simulating net218...
Done simulating net218 with 3 species and 1 reactions.

Simulating net219...
Done simulating net219 with 3 species and 1 reactions.

Simulating net220...
Done simulating net220 with 4 species and 2 reactions.

Simulating net221...
Done simulating net221 with 12 species and 11 reactions.

Simulating net222...
Done simulating net222 with 12 species and 9 reactions.

Simulating net223...
Done simulating net223 with 11 species and 11 reactions.

Simulating net224...
Done simulating net224 with 3 species and 1 reactions.

Simulating net225...
Done simulating net225 with 7 species and 3 reactions.

Simulating net226...
Done simulating net226 with 5 species and 2 reactions.

Simulating net227...
Done simulating net227 with 3 species and 1 reactions.

Simulating net228...
Done simulating net228 with 9 species and 4 reactions.

Simulating net229...
Done simulating net229 with 4 species and 2 reactions.

Simulating net2

Done simulating net324 with 6 species and 3 reactions.

Simulating net325...
Done simulating net325 with 11 species and 7 reactions.

Simulating net326...
Done simulating net326 with 5 species and 2 reactions.

Simulating net327...
Done simulating net327 with 2 species and 1 reactions.

Simulating net328...
Done simulating net328 with 5 species and 2 reactions.

Simulating net329...
Done simulating net329 with 5 species and 2 reactions.

Simulating net330...
Done simulating net330 with 3 species and 3 reactions.

Simulating net331...
Done simulating net331 with 12 species and 16 reactions.

Simulating net332...
Done simulating net332 with 7 species and 3 reactions.

Simulating net333...
Done simulating net333 with 8 species and 7 reactions.

Simulating net334...
Done simulating net334 with 8 species and 8 reactions.

Simulating net335...
Done simulating net335 with 9 species and 9 reactions.

Simulating net336...
Done simulating net336 with 12 species and 12 reactions.

Simulating net3

In [22]:
# Unite all networks into a single chemkin file
import re, string

#net=406

united = open('output/unitedChemkin.inp', 'w')
united.truncate(0)

original = open('input/chemkin.inp', 'r').readlines()

copy = True # copy the SPECIES and THERMO sections from the original file
lastline = ''
getlines = []
for line in original:
    if copy:
        getlines.append(line)
        if 'REACTIONS' in line: # copy the REACTIONS line, but then don't continue automaticaly
            copy = False
            getlines.append('\n')
    elif line[0] != '!' and 'END' not in line and line.count('+') == 3\
            and line.count('=') == 1 and line.count('+M') == 0: # copy only reactions which aren't unimolecular
        getlines.append(line)
        getlines.append('\n')
    elif "DUP" in line and lastline[0] != '!' and lastline.count('+') == 3\
            and lastline.count('=') == 1 and lastline.count('+M') == 0:
        getlines[-1] = line
        getlines.append('\n')
    lastline = line
united.writelines("%s" % str(item) for item in getlines)

for n in xrange(net):
    network = n + 1
    if network not in errored:
        f = open('runs/net'+str(network)+'/chemkin/chem_annotated.inp', 'r')
        species = False
        reactions = False
        #dictionary[] = {}
        dictionary['O2(2)'] = 'O2_2_(13)' # This is udapted per case
        comment = False # comment out using '!' reactions that contain 'CH4(1)'

        for line in f:
            if 'SPECIES' in line:
                species = True
            elif 'REACTIONS' in line:
                reactions = True
            elif 'END' in line:
                species = False
                reactions = False

            # make a dictionary of original species names (since RMG probably changed them)
            if species and 'SPECIES' not in line and 'CH4(1)' not in line and 'O2(2)' not in line:
                line=("".join(line.split())).split('!') # remove spaces and split at '!'
                if re.search('\(\d*\)$', line[1]) is not None:
                    dictionary[line[0]] = line[1][0:-len(re.search('\(\d*\)$', line[1]).group(0))]

            if reactions:
                for key in dictionary.keys():
                    line = string.replace(line,key,dictionary[key]) # replace species' names with dictionary names
                if 'CH4(1)' in line:
                    comment = True
                if line == '\n':
                    comment = False
                if comment:
                    line = '!' + line
                if 'REACTIONS' not in line:
                    united.writelines("%s" % line)
        f.close()
        
# still need to add all non-pdep
    
united.writelines("%s" % '\nEND')
united.close()

print("Saved PDep model into output directory.")
if len(errored) > 0:
    print("\n{0} networks resulted in an error:".format(len(errored)))
    for e in errored:
        print("ERROR in network {0}!".format(network))

Saved PDep model into output directory.
